# Galaxy SED Matching using selectGalaxySED.py

"selectGalaxySED.py" is designed to do the matching for galaxy objects. There are two steps to using this program: 

1) First load the model SEDs you would like to use. If using the Bruzual & Charlot models from sims_sed_library you can use the built-in "loadBC03" method provided as shown in the example below. However, this can be used as a template for creating loading methods for your own model SEDs if desired. Be aware that since the BC03 SEDs are defined in the sims_sed_library to be in the rest frame, this code assumes that any incoming model SEDs are in the rest frame. If using your own folder of model SEDs specify the parameter "galDir" when initializing the class.

In [ ]:
from lsst.sims.photUtils.selectGalaxySED import selectGalaxySED

#If you are using your own folder this should be selectGalaxySED(galDir = yourFolder)
matchSED = selectGalaxySED()
"""Can restrict the loading below to a subset of the SEDs in a given folder if you pass a list
as matchSED.loadBC03(subset = subsetList)"""
sedList = matchSED.loadBC03()

Now you have a set of SED class objects. Each object in the set has the attributes wavelen, flambda, name, type, age, and metallicity (defined as a ratio to solar metallicity).

In [ ]:
print sedList[0].wavelen
print sedList[0].flambda
print sedList[0].name

You now have a set of model SEDs ready to match to your catalog objects.

2) Once the SEDs are loaded you have two options for matching depending on your catalog type. If your catalog contains rest frame magnitudes and needs no dust or redshift corrections then you can match to the model SEDs using "matchToRestFrame". Before this you need to specify your bandpasses and pass them in to the rest frame matching algorithm.

Notice the matching algorithm also returns magNorm values that when used with the matched SED will
most closely reproduce the catalog object's magnitudes.

In [ ]:
import os
import numpy as np
import lsst.utils
from lsst.sims.photUtils import BandpassDict

#Import sample galaxy catalog organized as [RA, Dec, z, mag_g, mag_r, mag_i, mag_z, mag_y]
ra, dec, z, mag_g, mag_r, mag_i, mag_z, mag_y = np.genfromtxt('sampleCatalogs/sampleGalCat.dat', unpack=True)

"""Organize magnitudes into single array with each row corresponding to the 
set of magnitudes of one object"""
catMags = np.transpose([mag_g, mag_r, mag_i, mag_z, mag_y])

#Define bandpasses. This example shows loading in LSST grizy bandpasses.
galBandpassDict = BandpassDict.loadTotalBandpassesFromFiles(['g', 'r', 'i', 'z', 'y'])

#We'll need to set makeCopy to True since we are going to reuse this sedList below
matchRestFrameNames, matchRestMagNorm, matchErrors = matchSED.matchToRestFrame(sedList, catMags, 
                                                                               bandpassDict = galBandpassDict,
                                                                               makeCopy = True)

#Show the resulting matches
print matchRestFrameNames
print matchRestMagNorm
print matchErrors

If your catalog magnitudes are observed and require correction for milky way dust and redshift you should use "matchToObserved". 

Since for speed this generates colors for the model SEDs on a grid of redshift values, you can specify the digits after the decimal point you want to step through in this grid with the parameter dzAcc (default is dzAcc = 2 which will make a grid with steps in z equal to 0.01).

If correcting for reddening refer to <a href="http://adsabs.harvard.edu/abs/2011ApJ...737..103S">Schlafly and Finkbeiner (2011)</a> for the extinction coefficients for your filters. We use R<sub>v</sub> = 3.1 coefficients in Table 6 in that paper. Defaults are for SDSS ugriz. For reference:

SDSS ugriz coefficients in order are [4.239, 3.303, 2.285, 1.698, 1.263]

LSST ugrizy are [4.145, 3.237, 2.273, 1.684, 1.323, 1.088]

In [ ]:
#This time we will use SDSS griz bandpasses. 

#If using full SDSS ugriz then you do not have to load the Bandpasses ahead of time
#or specify the extCoeffs since these are the default values for matchToObserved.
sdssBandpassDict = BandpassDict.loadTotalBandpassesFromFiles(['g', 'r', 'i', 'z'], bandpassDir = os.path.join(lsst.utils.getPackageDir('throughputs'),'sdss'),
                                bandpassRoot ='sdss_')

catMagsObs = np.transpose([mag_g, mag_r, mag_i, mag_z])

matchObservedNames, matchObsMagNorm, matchObsErrors = matchSED.matchToObserved(sedList, catMagsObs, z, 
                                                                               catRA = ra, catDec = dec, 
                                                                               bandpassDict = sdssBandpassDict,
                                                                               dzAcc = 2,
                                                                               extCoeffs = (3.303, 2.285, 1.698, 
                                                                                            1.263))
print matchObservedNames
print matchObsMagNorm
print matchObsErrors